<a href="https://colab.research.google.com/github/vineet96/vertex-ai-class/blob/main/Text_classification_VerteXAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    google-cloud-storage \
                                    jsonlines

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
PROJECT_ID = "vertexai-dev-387420"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
BUCKET_NAME = "vertexai-dev-text-classification"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
! gsutil mb -l $REGION $BUCKET_URI

In [ ]:
import jsonlines
from google.cloud import aiplatform, storage
from google.cloud.aiplatform import jobs

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

Create a Dataset resource and import your data

In [ ]:
# # Use a timestamp to ensure unique resources
# src_uris = "gs://vertexai-dev-text-classification/data_happiness.csv"
# display_name = "text-classification-dataset"

# text_dataset = aiplatform.TextDataset.create(
#     display_name=display_name,
#     gcs_source=src_uris,
#     import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
#     sync=True,
# )

Access Dataset resource with Dateset_id

In [ ]:
text_dataset = aiplatform.TextDataset('6067806650486489088')

Create a Training job

In [ ]:
# training_job_display_name = "text-classification-model"
# job = aiplatform.AutoMLTextTrainingJob(
#     display_name=training_job_display_name,
#     prediction_type="classification",
#     multi_label=False,
# )

Run Training Job and Create the model

In [ ]:
# model_display_name = "text-classification-model"

# # Run the training job
# model = job.run(
#     dataset=text_dataset,
#     model_display_name=model_display_name,
#     training_fraction_split=0.8,
#     validation_fraction_split=0.1,
#     test_fraction_split=0.1,
#     sync=True,
# )

Access Model with model-id

In [ ]:
model = aiplatform.Model("projects/1052777806387/locations/us-central1/models/c12345678")

Deploy your model to an endpoint for online prediction

In [ ]:
# deployed_model_display_name = "text-classification-model"

# endpoint = model.deploy(
#     deployed_model_display_name=deployed_model_display_name, sync=True
# )

In [ ]:
endpoint = aiplatform.Endpoint('1026358920156807168')

Get Online prediction

In [ ]:
content = "I love my daughter"

response = endpoint.predict(instances=[{"content": content}])

for prediction_ in response.predictions:
    ids = prediction_["ids"]
    display_names = prediction_["displayNames"]
    confidence_scores = prediction_["confidences"]
    for count, id in enumerate(ids):
        print(f"Prediction ID: {id}")
        print(f"Prediction display name: {display_names[count]}")
        print(f"Prediction confidence score: {confidence_scores[count]}")

Create Batch Prediction

In [ ]:
instances = [
    "We hiked through the woods and up the hill to the ice caves",
    "My kitten is so cute",
]
input_file_name = "batch-prediction-input.jsonl"

In [ ]:
# Instantiate the Storage client and create the new bucket
# from google.cloud import  storage
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)
# Iterate over the prediction instances, creating a new TXT file
# for each.
input_file_data = []
for count, instance in enumerate(instances):
    instance_name = f"input_{count}.txt"
    instance_file_uri = f"{BUCKET_URI}/{instance_name}"
    # Add the data to store in the JSONL input file.
    tmp_data = {"content": instance_file_uri, "mimeType": "text/plain"}
    input_file_data.append(tmp_data)

    # Create the new instance file
    blob = bucket.blob(instance_name)
    blob.upload_from_string(instance)

input_str = "\n".join([str(d) for d in input_file_data])
file_blob = bucket.blob(f"{input_file_name}")

file_blob.upload_from_string(input_str)

In [ ]:
job_display_name = "e2e-text-classification-batch-prediction-job"
# model = aiplatform.Model(model_name=model.name)
batch_prediction_job = model.batch_predict(
    job_display_name=job_display_name,
    gcs_source="gs://vertexai-dev-text-classification/batch-prediction-input.jsonl",
    gcs_destination_prefix=f"{BUCKET_URI}/output",
    sync=True,
)
batch_prediction_job_name = batch_prediction_job.resource_name

Clean UP

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# Delete batch
batch_job.delete()

# Undeploy endpoint
endpoint.undeploy_all()

# `force` parameter ensures that models are undeployed before deletion
endpoint.delete()

# Delete model
model.delete()

# Delete text dataset
text_dataset.delete()

# Delete training job
job.delete()